### Predective Model Target Drug Eligibility

In [225]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
df_train = pd.read_parquet(r"train.parquet")
df_train.head(5)

,Patient-Uid,Date,Incident
0,a0db1e73-1c7c-11ec-ae39-16262ee38c7f,2019-03-09,PRIMARY_DIAGNOSIS
1,a0dc93f2-1c7c-11ec-9cd2-16262ee38c7f,2015-05-16,PRIMARY_DIAGNOSIS
3,a0dc94c6-1c7c-11ec-a3a0-16262ee38c7f,2018-01-30,SYMPTOM_TYPE_0
4,a0dc950b-1c7c-11ec-b6ec-16262ee38c7f,2015-04-22,DRUG_TYPE_0
8,a0dc9543-1c7c-11ec-bb63-16262ee38c7f,2016-06-18,DRUG_TYPE_1


In [226]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3220868 entries, 0 to 29080911
Data columns (total 3 columns):
 #   Column       Dtype         
---  ------       -----         
 0   Patient-Uid  object        
 1   Date         datetime64[ns]
 2   Incident     object        
dtypes: datetime64[ns](1), object(2)
memory usage: 98.3+ MB


#### Sort Each Patient With Date

In [227]:
df_train = df_train.sort_values(['Patient-Uid', 'Date']).reset_index(drop=True)
df_train[:5]

,Patient-Uid,Date,Incident
0,a0db1e73-1c7c-11ec-ae39-16262ee38c7f,2015-09-22,DRUG_TYPE_7
1,a0db1e73-1c7c-11ec-ae39-16262ee38c7f,2018-04-13,SYMPTOM_TYPE_2
2,a0db1e73-1c7c-11ec-ae39-16262ee38c7f,2018-05-02,DRUG_TYPE_7
3,a0db1e73-1c7c-11ec-ae39-16262ee38c7f,2018-11-23,SYMPTOM_TYPE_0
4,a0db1e73-1c7c-11ec-ae39-16262ee38c7f,2018-11-23,DRUG_TYPE_9


In [ ]:
# Find unique drugs and unique patients

In [81]:
unique_train_incidents=df_train['Incident'].unique()
unique_train_patients=df_train['Patient-Uid'].unique()
unique_train_incidents

array(['DRUG_TYPE_7', 'SYMPTOM_TYPE_2', 'SYMPTOM_TYPE_0', 'DRUG_TYPE_9',
       'DRUG_TYPE_0', 'TEST_TYPE_0', 'DRUG_TYPE_11', 'DRUG_TYPE_2',
       'PRIMARY_DIAGNOSIS', 'TEST_TYPE_1', 'DRUG_TYPE_8', 'DRUG_TYPE_6',
       'SYMPTOM_TYPE_7', 'DRUG_TYPE_3', 'SYMPTOM_TYPE_6',
       'SYMPTOM_TYPE_17', 'DRUG_TYPE_1', 'SYMPTOM_TYPE_3',
       'SYMPTOM_TYPE_8', 'SYMPTOM_TYPE_15', 'TEST_TYPE_2',
       'SYMPTOM_TYPE_10', 'TEST_TYPE_4', 'SYMPTOM_TYPE_24',
       'SYMPTOM_TYPE_4', 'DRUG_TYPE_5', 'DRUG_TYPE_10', 'SYMPTOM_TYPE_1',
       'SYMPTOM_TYPE_9', 'DRUG_TYPE_12', 'SYMPTOM_TYPE_5', 'TEST_TYPE_3',
       'DRUG_TYPE_15', 'DRUG_TYPE_4', 'DRUG_TYPE_14', 'DRUG_TYPE_13',
       'SYMPTOM_TYPE_16', 'SYMPTOM_TYPE_14', 'SYMPTOM_TYPE_18',
       'SYMPTOM_TYPE_13', 'SYMPTOM_TYPE_29', 'SYMPTOM_TYPE_11',
       'SYMPTOM_TYPE_12', 'SYMPTOM_TYPE_22', 'SYMPTOM_TYPE_19',
       'DRUG_TYPE_16', 'DRUG_TYPE_17', 'SYMPTOM_TYPE_21', 'TEST_TYPE_5',
       'SYMPTOM_TYPE_20', 'SYMPTOM_TYPE_26', 'SYMPTOM_TYPE_27',
   

### Create new colum for every drug and put its frequency

In [ ]:
import warnings
warnings.filterwarnings("ignore")
# Health_df=pd.DataFrame()
for pid in unique_train_patients:
    pid_row={}
    pid_history = df_train[df_train['Patient-Uid'] == pid]   
    pid_row['PID']=pid
    pid_incidents=list(pid_history['Incident'])
    for incident in sorted(unique_train_incidents):
        count=0
        count=pid_incidents.count(incident)
        pid_row[incident]=count
    if 'TARGET DRUG' in pid_incidents:
        pid_row['ELIGIBILITY']=1
    else:
        pid_row['ELIGIBILITY']=0
    Health_df = Health_df.append(pid_row, ignore_index = True)

In [138]:
Health_df.head(2)

,PID,DRUG_TYPE_0,DRUG_TYPE_1,DRUG_TYPE_10,DRUG_TYPE_11,DRUG_TYPE_12,DRUG_TYPE_13,DRUG_TYPE_14,DRUG_TYPE_15,DRUG_TYPE_16,...,SYMPTOM_TYPE_8,SYMPTOM_TYPE_9,TARGET DRUG,TEST_TYPE_0,TEST_TYPE_1,TEST_TYPE_2,TEST_TYPE_3,TEST_TYPE_4,TEST_TYPE_5,ELIGIBILITY
0,a0db1e73-1c7c-11ec-ae39-16262ee38c7f,29,0,0,1,0,0,0,0,0,...,0,0,0,10,2,0,0,0,0,0
1,a0dc93f2-1c7c-11ec-9cd2-16262ee38c7f,8,27,0,0,0,0,0,0,0,...,0,0,0,1,4,0,0,0,0,0


In [228]:
# Data set for training Model
trainig=Health_df.copy()
del trainig['ELIGIBILITY']
del trainig['PID']
del trainig['TARGET DRUG']
y=Health_df['ELIGIBILITY']
X=trainig
X = np.array(X)  
y = np.array(y).reshape(-1)

In [167]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
import numpy as np

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# RandomForestClassifier
model = RandomForestClassifier(n_estimators=150, max_depth=20, min_samples_split=5, 
                               min_samples_leaf=3, max_features='auto', random_state=42)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
confusion_mat = confusion_matrix(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Confusion Matrix:")
print(confusion_mat)
print("F1 Score:", f1)


Accuracy: 0.8077681874229347
Confusion Matrix:
[[4667  615]
 [ 944 1884]]
F1 Score: 0.7073399662098743


### final_submission.csv

In [ ]:
# read the test file and need to apply all the steps that are applied during the training process

In [175]:
df_test = pd.read_parquet(r"test.parquet")
df_test = df_test.sort_values(['Patient-Uid', 'Date']).reset_index(drop=True)
df_test.to_csv('Patient_test.csv')
df_test=pd.read_csv(r"Patient_test.csv")
unique_test_incidents=df_test['Incident'].unique()
unique_test_patients=df_test['Patient-Uid'].unique()

In [ ]:
# create all the rows that are created during training process

In [180]:
# Health_test_df=pd.DataFrame()
for pid in unique_test_patients:
    pid_row={}
    pid_history = df_test[df_test['Patient-Uid'] == pid]   
    pid_row['PID']=pid
    pid_incidents=list(pid_history['Incident'])
    for incident in sorted(unique_train_incidents):
        count=0
        count=pid_incidents.count(incident)
        pid_row[incident]=count
    Health_test_df = Health_test_df.append(pid_row, ignore_index = True)

In [229]:
Health_test_df[:5]

,PID,DRUG_TYPE_0,DRUG_TYPE_1,DRUG_TYPE_10,DRUG_TYPE_11,DRUG_TYPE_12,DRUG_TYPE_13,DRUG_TYPE_14,DRUG_TYPE_15,DRUG_TYPE_16,...,SYMPTOM_TYPE_7,SYMPTOM_TYPE_8,SYMPTOM_TYPE_9,TARGET DRUG,TEST_TYPE_0,TEST_TYPE_1,TEST_TYPE_2,TEST_TYPE_3,TEST_TYPE_4,TEST_TYPE_5
0,a0f9e8a9-1c7c-11ec-8d25-16262ee38c7f,8,3,0,1,0,0,0,0,0,...,0,0,0,0,2,0,0,0,0,0
1,a0f9e9f9-1c7c-11ec-b565-16262ee38c7f,2,30,0,0,0,0,0,9,0,...,0,0,0,0,0,0,0,1,0,0
2,a0f9ea43-1c7c-11ec-aa10-16262ee38c7f,4,33,0,0,0,0,0,0,0,...,0,0,0,0,0,2,0,0,0,0
3,a0f9ea7c-1c7c-11ec-af15-16262ee38c7f,2,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,a0f9eab1-1c7c-11ec-a732-16262ee38c7f,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [200]:
Health_test_df.head(2)
final_submission=Health_test_df.copy()
del final_submission['TARGET DRUG']
del final_submission['PID']

In [201]:
# predict the test set with the model
y_pred = model.predict(final_submission)

In [203]:
final_submission['Patient-Uid']=Health_test_df['PID']
final_submission['label']=y_pred
final_submission=final_submission[['Patient-Uid', 'label']]

In [222]:
final_submission.to_csv('my_final_submission.csv')